<a href="https://colab.research.google.com/github/ragingthunder511/da6401_assignment2/blob/main/cs24m020_dl_a1_partB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install wandb
import wandb
# !wandb login
wandb.login(key="01bb56b62b8d93215a878ebdbc41b79e456d010c")

#01bb56b62b8d93215a878ebdbc41b79e456d010c

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karekargrishma1234 (karekargrishma1234-iit-madras-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip -O nature_12K.zip
!unzip -q nature_12K.zip
!rm nature_12K.zip

--2025-04-19 15:19:20--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.207, 142.251.18.207, 108.177.119.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  38.9MB/s    in 91s     

2025-04-19 15:20:52 (39.9 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import wandb

# Use your best config
BEST_CONFIG = {
    'weight_decay': 0,
    'learning_rate': 1e-4,
    'dropout': 0.4,
    'activation': 'relu',
    'optimiser': 'rmsprop',
    'batch_norm': 'true',
    'batch_size': 32,
    'dense_layer': 256
}

# Initialize wandb
wandb.init(project='cs24m020_dl_a2_partB', config=BEST_CONFIG)
config = wandb.config

# Transforms to match ImageNet pretrained input format
common_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Load the full dataset from train folder
full_dataset = datasets.ImageFolder('inaturalist_12K/train', transform=train_transforms)

# Split into 80% train and 20% validation
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Apply non-augmented transforms to validation dataset
val_dataset.dataset.transform = common_transforms

# Load test dataset from 'val' folder (used as test set)
test_dataset = datasets.ImageFolder('inaturalist_12K/val', transform=common_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

# Load pretrained model
model = models.resnet50(pretrained=True)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Activation function
activation_fn = nn.ReLU() if config.activation == 'relu' else nn.Tanh()

# Build classifier head
head_layers = [
    nn.Dropout(config.dropout),
    nn.Linear(model.fc.in_features, config.dense_layer)
]
if config.batch_norm == 'true':
    head_layers.append(nn.BatchNorm1d(config.dense_layer))
head_layers.extend([
    activation_fn,
    nn.Linear(config.dense_layer, 10)
])
model.fc = nn.Sequential(*head_layers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
if config.optimiser == 'rmsprop':
    optimizer = optim.RMSprop(model.fc.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
elif config.optimiser == 'adam':
    optimizer = optim.Adam(model.fc.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
else:
    optimizer = optim.SGD(model.fc.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

# Train and validate functions
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return total_loss / len(loader), correct / total

# Training loop
EPOCHS = 10
for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    train_acc *= 100
    val_acc *= 100

    print(f"Epoch {epoch+1}:Train Loss={train_loss:.4f},Train Acc={train_acc:.4f},Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}")
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_accuracy': train_acc,
        'val_loss': val_loss,
        'val_accuracy': val_acc
    })

# Test evaluation
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
test_acc *= 100

print(f"\n✅ Test Loss: {test_loss:.4f}")
print(f"✅ Test Accuracy: {test_acc:.4f}")

wandb.log({
    'test_loss': test_loss,
    'test_accuracy': test_acc
})

# Save model
torch.save(model.state_dict(), "finetuned_resnet50.pth")
wandb.save("finetuned_resnet50.pth")
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 190MB/s]


Epoch 1:Train Loss=1.1989,Train Acc=64.8581,Val Loss=0.9094, Val Acc=73.4000
Epoch 2:Train Loss=0.9262,Train Acc=71.2464,Val Loss=0.8254, Val Acc=74.1500
Epoch 3:Train Loss=0.8527,Train Acc=72.3215,Val Loss=0.7921, Val Acc=75.1000
Epoch 4:Train Loss=0.8044,Train Acc=73.9342,Val Loss=0.7652, Val Acc=75.9500
Epoch 5:Train Loss=0.7749,Train Acc=74.2718,Val Loss=0.7513, Val Acc=76.0500
Epoch 6:Train Loss=0.7524,Train Acc=74.7093,Val Loss=0.7486, Val Acc=75.2000
Epoch 7:Train Loss=0.7339,Train Acc=75.3094,Val Loss=0.7540, Val Acc=75.7000
Epoch 8:Train Loss=0.7300,Train Acc=75.8470,Val Loss=0.7373, Val Acc=76.4500
Epoch 9:Train Loss=0.6966,Train Acc=76.9471,Val Loss=0.7332, Val Acc=76.1000
Epoch 10:Train Loss=0.6923,Train Acc=76.6346,Val Loss=0.7322, Val Acc=76.0500

✅ Test Loss: 0.6791
✅ Test Accuracy: 78.0000


epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▁▅▅▆▆▇▇▇██
train_loss,█▄▃▃▂▂▂▂▁▁
val_accuracy,▁▃▅▇▇▅▆█▇▇
val_loss,█▅▃▂▂▂▂▁▁▁
epoch,10
test_accuracy,78
test_loss,0.67914
train_accuracy,76.63458
